In [1]:
import imageio
import torch
import torchvision

from PIL import Image
import requests
from io import BytesIO

import cv2

In [15]:
device = "cuda:0"
m = torch.load('weights/densenet121-epoch-10-val_acc=0.9952-test_acc=-1.00.pt')
m = m.to(device)
m.eval()

tr = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=(0.4005, 0.3702, 0.3419), std=(0.2858, 0.2749, 0.2742)),
    ]
)

In [16]:
filename = '/media/tomek/BIG2/datasets/FIRE/test_burn/DJI_0002.MOV'
reader = imageio.get_reader(filename,  'ffmpeg')
fps = reader.get_meta_data()['fps']
writer = imageio.get_writer('out.mp4', fps=fps)

In [19]:
!pwd

/home/tomek/pro/fire-detect-nn


In [ ]:
# get the gradient of the output with respect to the parameters of the model
pred[:, 386].backward()

# pull the gradients out of the model
gradients = vgg.get_activations_gradient()

# pool the gradients across the channels
pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])

# get the activations of the last convolutional layer
activations = vgg.get_activations(img).detach()

# weight the channels by corresponding gradients
for i in range(512):
    activations[:, i, :, :] *= pooled_gradients[i]
    
# average the channels of the activations
heatmap = torch.mean(activations, dim=1).squeeze()

# relu on top of the heatmap
# expression (2) in https://arxiv.org/pdf/1610.02391.pdf
heatmap = np.maximum(heatmap, 0)

# normalize the heatmap
heatmap /= torch.max(heatmap)

# draw the heatmap
plt.matshow(heatmap.squeeze())

In [17]:
score = 1
a = 0.1
for arr in reader.iter_data():
    img = Image.fromarray(arr)
    tensor_in = tr(img).to(device)
    batch_in = torch.unsqueeze(tensor_in, dim=0)
    batch_out = m(batch_in)

    score = 0.9 * score + 0.1 * (1-float(batch_out[0]))
    cv2.putText(
     arr, #numpy array on which text is written
     f"Fire score: {score}", #text
     (100,1080-100), #position at which writing has to start
     cv2.FONT_HERSHEY_SIMPLEX, #font family
     2, #font size
     (255, 255, 255, 255), #font color
     3) #font stroke
    
    writer.append_data(arr)
    
writer.close()

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1920, 1080) to (1920, 1088) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [14]:
writer.close()